In [384]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv ('C:/Users/garim/Documents/Projects/Data Mining Project/ted_main.csv')
print (data.columns)
data['film_date'] =data['film_date'].apply(lambda x: dt.datetime.fromtimestamp( int(x)).strftime('%d-%m-%Y'))
data['published_date'] = data['published_date'].apply(lambda x: dt.datetime.fromtimestamp( int(x)).strftime('%d-%m-%Y'))

Index(['comments', 'description', 'duration', 'event', 'film_date',
       'languages', 'main_speaker', 'name', 'num_speaker', 'published_date',
       'ratings', 'related_talks', 'speaker_occupation', 'tags', 'title',
       'url', 'views'],
      dtype='object')


In [385]:
#Calculating quartile ranges and means
data = data.sort_values('views')
mid = int(len(data)/2)
k=list(data['views'])
lowerQ = np.mean(sorted(k[:mid]))
mean = np.mean(sorted(k))
upperQ = np.mean(sorted(k[mid:]))
print(lowerQ)
print(mean)
print(upperQ)

729813.14902
1698297.48157
2666781.81412


In [386]:
#Add a new column views_based
views_based=""
def views_function(data_views):
        if (data_views<lowerQ):
            views_based='Poor'
        if (data_views>lowerQ and data_views<mean):
            views_based='Moderate'
        if (data_views<upperQ and data_views>mean):
            views_based='High'
        if (data_views>upperQ):
            views_based='Very High'
        return (views_based) 
data["views_based"] = data['views'].apply(views_function)

In [387]:
import ast
data['ratings'] = data['ratings'].apply(lambda x: ast.literal_eval(x))


In [388]:
#Add Topic column in the dataset
talk_topic = ""
def topic_function(data_topic):
    data_topic = sorted(data_topic, key=lambda k: k['count'])
    return data_topic[0]['name']
data['Topics']= data['ratings'].apply(topic_function)

In [389]:
pos_or_neg = ""
def pos_neg_function(rating):
    i=rating
    if (i=='Beautiful' or i=='Courageous' or i=='Funny' or i=='Informative' or i=='Ingenious' or i=='Inspiring' or i=='Jaw-dropping' or i=='Persuasive' ):
        pos_or_neg='Positive'
    elif (i=='OK'):
        pos_or_neg='Neutral'
    else :
        pos_or_neg='Negative'
    return pos_or_neg
data["comment_rating"] = data['Topics'].apply(pos_neg_function)

In [390]:
#Add year as a column
#data['film_date'] = pd.to_datetime(data['film_date'])
#data['film_year'] = data['film_date'].dt.year.astype(int)
#data['published_date'] = pd.to_datetime(data['published_date'])
#data['published_year'] = data['published_date'].dt.year.astype(int)

#transform(data, film_year = as.numeric(film_year))

In [391]:
#classification for num_of_speakers
from sklearn.model_selection import train_test_split
predictors = data.drop(['views_based','Topics','published_date','film_date',"comment_rating",'comments','description','event','main_speaker','name','ratings','related_talks','speaker_occupation','tags','title','url'], axis=1)
target = data["num_speaker"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 1)

In [392]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gaussian)

98.75


In [393]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_svc)

98.75


In [394]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

98.75


In [395]:
# Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
acc_linear_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_linear_svc)

98.75


In [396]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_decisiontree = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_decisiontree)

100.0


In [397]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

100.0


In [398]:
# KNN or k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_knn)

98.75


In [399]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_logreg, 
              acc_randomforest, acc_gaussian,acc_linear_svc, acc_decisiontree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
6,Decision Tree,100.00
0,Support Vector Machines,98.75
1,KNN,98.75
2,Logistic Regression,98.75
4,Naive Bayes,98.75
5,Linear SVC,98.75


In [400]:
#classification for views_based
from sklearn.model_selection import train_test_split
predictors = data.drop(['views_based','Topics','published_date','film_date',"comment_rating",'comments','description','event','main_speaker','name','ratings','related_talks','speaker_occupation','tags','title','url'], axis=1)
target = data["views_based"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 1)

In [401]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gaussian)

95.72


In [402]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_svc)

51.52


In [403]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

68.45


In [404]:
# Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
acc_linear_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_linear_svc)

60.25


In [405]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_decisiontree = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_decisiontree)

99.64


In [406]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

99.47


In [407]:
# KNN or k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_knn)

99.47


In [408]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_logreg, 
              acc_randomforest, acc_gaussian, acc_decisiontree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
5,Decision Tree,99.64
1,KNN,99.47
3,Random Forest,99.47
4,Naive Bayes,95.72
2,Logistic Regression,68.45
0,Support Vector Machines,51.52


In [409]:
#classification for comments
from sklearn.model_selection import train_test_split
predictors = data.drop(['views_based','Topics','published_date','film_date',"comment_rating",'comments','description','event','main_speaker','name','ratings','related_talks','speaker_occupation','tags','title','url'], axis=1)
target = data["comment_rating"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 1)

In [410]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gaussian)

48.84


In [411]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_svc)

64.17


In [412]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

64.17


In [413]:
#Linear SVC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
acc_linear_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_linear_svc)

64.17


In [414]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_decisiontree = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_decisiontree)

56.33


In [415]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

60.96


In [416]:
# KNN or k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_knn)

58.11


In [417]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_logreg, 
              acc_randomforest, acc_gaussian,acc_linear_svc, acc_decisiontree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,Support Vector Machines,64.17
2,Logistic Regression,64.17
5,Linear SVC,64.17
3,Random Forest,60.96
1,KNN,58.11
6,Decision Tree,56.33
4,Naive Bayes,48.84
